# 4. Procesado de datos.

## 4.2 Creación de conjunto de entrenamiento y pruebas. 

In [1]:
# Importando librerias.
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Importando datasets from datafiles.
mainpath = "/proyectos/learning/MachineLearning/datasets/aprendizaje-autom-profundo/"
outputs = "/proyectos/learning/MachineLearning/datasets/aprendizaje-autom-profundo/outputs/"
filename = "precios_casas.csv"
fullpath = os.path.join(mainpath, filename)

In [14]:
# Creando conjunto con biblioteca Scikit-learn funcion train_test_split().

"""
X = atributos descriptivos.
y = valores de la etiqueta.
entX = conjunto entrenamiento atributos descriptivos (3.680 x 16)
pruX = conjunto entrenamiento atributo etiqueta (3.680)
enty = conjunto de prueba atributos descriptivos (920 x 16)
pruy = conjunto de prueba atributo etiqueta (920)
test_size=0.2 = 20%
random_state=100 = Se usa como semilla para inicializar un objeto generador de numeros aleatorios.
"""

import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv(fullpath)
X = df.iloc[:,1:].values
y = df.iloc[:,0].values
print(X)
entX, pruX, enty, pruy = train_test_split(X, y, test_size=0.2, random_state=100)

[[3.0 1.5 1340 ... 'Shoreline' 'WA 98133' 'USA']
 [5.0 2.5 3650 ... 'Seattle' 'WA 98119' 'USA']
 [3.0 2.0 1930 ... 'Kent' 'WA 98042' 'USA']
 ...
 [3.0 2.5 3010 ... 'Renton' 'WA 98059' 'USA']
 [4.0 2.0 2090 ... 'Seattle' 'WA 98178' 'USA']
 [3.0 2.5 1490 ... 'Covington' 'WA 98042' 'USA']]
